In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 10
trials =1000

c1 = .005
c2 = .0025

adapt_c = False

# a  = .25, k= 9

In [5]:
a = 0.25
num_params=9


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 2.999935159332606, 4.063094856499803, 4.9127292647310465)


In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.80 & 0.70 & 0.80 & 0.90   \\
Model 1 & 0.20 & 0.20 & 0.20 & 0.30 & 0.20 & 0.10   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.7, 0.3, 0. ]), array([0.8, 0.2, 0. ]), array([0.9, 0.1, 0. ]), 8.970068202286233, 5.32496974650199, 7.151191205487481)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 1.0844246992922624, 2.9806182385708686, 3.5361767851954786)


# a  = .25, k= 4, something not right?

In [9]:
a = 0.25
num_params=4


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 0.90 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.10 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.9, 0.1, 0. ]), array([1., 0., 0.]), array([1., 0., 0.]), 0.8739202963303313, 6.041066748686283, 5.543315683679376)


# a  = .25, K = 19

In [11]:
a = 0.25
num_params=19


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 1.00 & 0.90 & 0.90 & 1.00   \\
Model 1 & 0.10 & 0.10 & 0.00 & 0.10 & 0.10 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.9, 0.1, 0. ]), array([0.9, 0.1, 0. ]), array([1., 0., 0.]), array([0.9, 0.1, 0. ]), array([0.9, 0.1, 0. ]), array([1., 0., 0.]), 3.056502804958305, 2.7113585424614954, 4.259502698107925)


# evidence of power

In [13]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [14]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.60 & 0.60 & 0.80 & 0.40 & 0.70 & 0.70   \\
Model 1 & 0.40 & 0.40 & 0.20 & 0.60 & 0.30 & 0.30   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.6, 0.4, 0. ]), array([0.6, 0.4, 0. ]), array([0.8, 0.2, 0. ]), array([0.4, 0.6, 0. ]), array([0.7, 0.3, 0. ]), array([0.7, 0.3, 0. ]), 9.54872395564911, 4.399042039817816, 5.083719029254441)


In [15]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.80 & 0.80 & 0.80 & 0.70 & 0.80 & 0.80   \\
Model 1 & 0.20 & 0.20 & 0.20 & 0.30 & 0.20 & 0.20   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), array([0.7, 0.3, 0. ]), array([0.8, 0.2, 0. ]), array([0.8, 0.2, 0. ]), 7.327201013638576, 4.107207658655526, 4.824404306946965)


In [ ]:
a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

# evidence of power 2

In [ ]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)